In [ ]:
#|default_exp download_rasters

# Download Rasters 

> utilities for conveniently downloading rasters

In [ ]:
#| eval: false
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import requests
import json
from fastcore.all import L
import pandas as pd
from typing import Any
from urllib.parse import urlparse
from pathlib import Path
import numpy as np
import re
from shapely.geometry import Polygon, box
from shapely import wkt
import geopandas as gpd
import sqlite3 as sql3
from pandas.io import sql 
import sqlalchemy as sqalc
import geowrangler.area_zonal_stats as azs
from fastdownload import FastDownload
from fastcore.foundation import Config
from functools import partial

In [ ]:
#| exporti
from ntlights_processing_utils.fetch_coords import search_dates_aoi 
from ntlights_processing_utils.eval_overlap import compute_overlaps

In [ ]:
#| eval: false
aoi = gpd.read_file('../data/admin_areas/ph_admin_bbox_wgs84.geojson')

In [ ]:
#| export
RASTER_TYPES = [
    'rade9.co.tif',
    'vflag.co.tif',
    'rad.co.tif',
    'samples.co.tif',
    'li.co.tif']

In [ ]:
%%time
#| eval: false
phl_matched20180501 = search_dates_aoi(aoi, date_param='20180501')

CPU times: user 44.3 ms, sys: 986 ms, total: 1.03 s
Wall time: 1.03 s


In [ ]:
%%time
#| eval: false
sorted_results = compute_overlaps(aoi,phl_matched20180501) 

CPU times: user 56.6 ms, sys: 2.6 ms, total: 59.2 ms
Wall time: 55.6 ms


In [ ]:
#| eval: false
cfg = Config('../data/ntlights-bigfiles','ntlights.ini',save=False)
fdownload = FastDownload(cfg, base='../data/ntlights-bigfiles',archive=folder)

In [ ]:
#| eval: false
folder, vflag_href, image_href = sorted_results.iloc[0].folder, sorted_results.iloc[0].vflag_href, sorted_results.iloc[0].asset_href

In [ ]:
#| eval: false
fdownload.download(vflag_href)

Path('/home/butch2/play/devpost/ntlights-processing-utils/nbs/../data/ntlights-bigfiles/npp_201805/npp_d20180501_t1639558_e1645362_b33728.vflag.co.tif')

In [ ]:
#| export

PATH_COLUMNS = {
    'rade9.co.tif':'image_path',
    'vflag.co.tif':'vflag_path',
    'rad.co.tif':'rad_path',
    'samples.co.tif':'samples_path',
    'li.co.tif':'li_path'
}
def get_path_column(raster_type):
    return PATH_COLUMNS[raster_type]
    

In [ ]:
#| export

def get_href_path(o, raster_type):
    if raster_type == 'rade9.co.tif':
        return o.asset_href
    elif raster_type == 'vflag.co.tif':
        return o.vflag_href
    else:
        return f'{o.base_url}/{o.folder}/{o.stem}.{raster_type}'

In [ ]:
#| export

def download_path_column(o, raster_type, dest):
    cfg = Config(dest,'ntlights.ini', save=False)
    fdownload = FastDownload(cfg,base=dest,archive=o.folder)
    url = get_href_path(o, raster_type)
    return fdownload.download(url)
    

In [ ]:
#| export

def download_items(items_df,dest='data', raster_type='rade9.co.tif'):
    if raster_type not in RASTER_TYPES:
        raise ValueError(f'Unknown raster_type: {raster_type}, valid values are {RASTER_TYPES}')
    path_column = get_path_column(raster_type) 
    download_func = partial(download_path_column, raster_type=raster_type, dest=dest)
    items_df[path_column] = items_df.apply(download_func, axis=1)
    return items_df
                         

In [ ]:
#| eval: false
top2_items = sorted_results[:2].copy()

In [ ]:
#| eval: false
top2_items = download_items(top2_items, dest='../data/ntlights-bigfiles')

In [ ]:
#| eval: false
top2_items.columns.values

array(['href', 'item_href', 'stem', 'suffix', 'base_url', 'folder',
       'product_id', 'start_date', 'first_scantime', 'end_scantime',
       'orbital_nbr', 'create_datetime', 'data_origin', 'data_domain',
       'vflag_file', 'vflag_href', 'cache_stem', 'asset_href', 'geometry',
       'section_area', 'intersect_area_sum', 'pct_covered', 'image_path'],
      dtype=object)